## Chatbots With Langgraph

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")
llm

## Start Building Chatbot Using Langgraph

In [ ]:
from typing import Annotated # for type hinting used in graph states

from typing_extensions import TypedDict # for creating custom types
from langgraph.graph import StateGraph,START,END # core langraph components for state management
from langgraph.graph.message import add_messages # to add messages to the graph

In [ ]:
# Defining the state structure using TypedDict of the graph
# Inheriting from TypedDict allows us to define a dictionary-like structure
class State(TypedDict):
  # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
  messages:Annotated[list,add_messages]

# Initializing the state graph with the defined state structure
graph_builder=StateGraph(State)


In [ ]:
graph_builder

In [ ]:
# Defining the chatbot function that takes the current state and returns the updated state
# The function uses the LLM to generate a response based on the messages in the state
def chatbot(state:State):
  return {"messages":llm.invoke(state['messages'])}

In [ ]:
# Adding the chatbot function as a node in the state graph
# name of the node is "chatbot" and the function is `chatbot`
graph_builder.add_node("chatbot",chatbot)

In [ ]:
graph_builder

In [ ]:
# Defining the edges of the graph to establish the flow
# The flow starts from the START node, goes to the "chatbot" node, and then to the END node
graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot",END)

In [ ]:
# Compiling the graph to finalize its structure
graph=graph_builder.compile()

In [ ]:
# Visualizing the graph using Mermaid syntax
from IPython.display import Image, display
try:
  display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  pass

In [ ]:
while True:
  user_input=input("User: ")

  # Exit condistion
  if user_input.lower() in ["quit","q"]:
    print("Good Bye")
    break

  # Streaming the graph execution with the user input as the initial message
  for event in graph.stream({'messages':("user",user_input)}):
    # Printing all event values from the graph execution - for debugging purposes
    print(event.values())

    # Printing only the assistant's response from the event values
    for value in event.values():
      print(value['messages'])
      print("Assistant:",value["messages"].content)